In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

# Intialize Plots w/ default fig width/height
try
  fig_width = 7
  fig_height = 5
  fig_format = :png
  fig_dpi = 96
  # no retina format type, use svg for high quality type/marks
  if fig_format == :retina
    fig_format = :svg
  # IJulia doesn't support PDF output so use png (if the DPI 
  # remains the default of 300 then set to 96)
  elseif fig_format == :pdf
    fig_format = :png
    fig_dpi = 96
  end
  # convert inches to pixels
  fig_width = fig_width * fig_dpi
  fig_height = fig_height * fig_dpi
  using Plots
  gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| code-fold: false
using VPL

struct A <: Node end

struct C <: Node
    val::Float64
end

struct B <: Node
    ID::Int
end

WebIO._IJuliaInit()

In [3]:
#| code-fold: false
motif(n, i = 0) = A() + (C(45.0) + A() + (C(45.0) +  A() + B(i + 1), 
                                           C(-45.0) + A() + B(i + 2), 
                                                       A() + B(i + 3)), 
                         C(- 45.0) + sum(A() for i in 1:n) + B(i + 4))
axiom =  motif(3, 0) + motif(2, 4) + motif(1, 8) + A() + A() + B(13)
graph = Graph(axiom);

In [4]:
#| code-fold: false
VPL.get_id(node_id, data::A) = "A"
VPL.get_id(node_id, data::C) = "C"*ifelse(sign(data.val) > 0.0, "+", "-")
VPL.get_id(node_id, data::B) = "B("*string(data.ID)*")"

In [5]:
#| code-fold: false
draw(graph)

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`'C:\Users\moral\.julia\packages\Blink\mwJC9\deps\atom\electron.exe' 'C:\Users\moral\.julia\packages\Blink\mwJC9\src\AtomShell\main.js' port 7957`, ProcessRunning), Sockets.TCPSocket(Base.Libc.WindowsRawSocket(0x0000000000000758) active, 0 bytes waiting), Dict{String, Any}("callback" => Blink.var"#1#2"())), Blink.Page(1, WebSocket(server, CONNECTED), Dict{String, Any}("webio" => Blink.AtomShell.var"#22#23"{Blink.AtomShell.WebIOBlinkComm}(Blink.AtomShell.WebIOBlinkComm(Blink.AtomShell.Window(#= circular reference @-5 =#))), "callback" => Blink.var"#1#2"()), Distributed.Future(1, 1, 1, ReentrantLock(nothing, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), 0), Some(true))), Task (done) @0x0000000078360f90)

In [6]:
#| code-fold: false
Q1 = Query(B)

Query object for nodes of type B


In [7]:
#| code-fold: false
A1 = apply(graph, Q1)

13-element Vector{B}:
 B(7)
 B(8)
 B(3)
 B(10)
 B(5)
 B(6)
 B(2)
 B(12)
 B(1)
 B(11)
 B(13)
 B(9)
 B(4)

In [8]:
function Q2_fun(n)
    check, steps = hasAncestor(n, x -> data(x) isa C)
    !check
end

Q2_fun (generic function with 1 method)

In [9]:
Q2 = Query(B, Q2_fun)
A2 = apply(graph, Q2)

1-element Vector{B}:
 B(13)

In [10]:
function branch_motif(p)
    data(p) isa A && 
    hasDescendent(p, x -> data(x) isa C && data(x).val < 0.0) &&
    hasAncestor(p, x -> data(x) isa C && data(x).val > 0.0)[1]
end

function Q3_fun(n, nsteps)
    # Condition 1
    check, steps = hasAncestor(n, branch_motif)
    !check && return false
    # Condition 2
    p = parent(n, steps)
    check, steps = hasAncestor(p, isRoot)
    steps != nsteps && return false
    return true
end

Q3_fun (generic function with 1 method)

In [11]:
Q3 = Query(B, n -> Q3_fun(n, 2))
A3 = apply(graph, Q3)

3-element Vector{B}:
 B(3)
 B(2)
 B(1)

In [12]:
function Q4_fun(n)
    !hasAncestor(n, x -> !isRoot(x) && length(children(x)) > 1)[1]
end

Q4_fun (generic function with 1 method)

In [13]:
Q4 = Query(B, Q4_fun)
A4 = apply(graph, Q4)

1-element Vector{B}:
 B(4)

In [14]:
function Q5_fun(n)
    check, steps = hasAncestor(n, isRoot)
    steps == 4
end

Q5 = Query(B, Q5_fun)
A5 = apply(graph, Q5)

1-element Vector{B}:
 B(3)

In [15]:
function Q6_fun(n, nA)
    check = Q3_fun(n, nA)
    !check && return false
    p2 = parent(n,2)
    data(p2) isa A
end

Q6 = Query(B, n -> Q6_fun(n, 3))
A6 = apply(graph, Q6)

1-element Vector{B}:
 B(7)

In [16]:
Q7 = Query(B, n -> Q6_fun(n, 4) || Q2_fun(n))
A7 = apply(graph, Q7)

2-element Vector{B}:
 B(11)
 B(13)

In [17]:
function Q8_fun(n)
    p1 = parent(n)
    p2 = parent(n, 2)
    p3 = parent(n, 3)
    data(p1) isa A && data(p2) isa C && data(p2).val > 0.0 && data(p3) isa A
end

Q8 = Query(B, Q8_fun)
A8 = apply(graph, Q8)

3-element Vector{B}:
 B(5)
 B(1)
 B(9)

In [18]:
function Q9_fun(n)
    p1 = parent(n)
    p2 = parent(n, 2)
    p3 = parent(n, 3)
    p4 = parent(n, 4)
    data(p1) isa A && data(p2) isa C && data(p2).val < 0.0 && 
       data(p3) isa A && data(p4) isa C
end

Q9 = Query(B, Q9_fun)
A9 = apply(graph, Q9)

3-element Vector{B}:
 B(10)
 B(6)
 B(2)

In [19]:
function Q10_fun(n)
    Q6_fun(n, 3) && return true # Check node 7
    Q9_fun(n) && hasAncestor(n, isRoot)[2] == 6 && return true # Check node 6
    hasAncestor(n, isRoot)[2] == 5 && data(parent(n, 3)) isa C && return true # Check node 8 (and not 4!)
end

Q10 = Query(B, Q10_fun)
A10 = apply(graph, Q10)

3-element Vector{B}:
 B(7)
 B(8)
 B(6)

In [20]:
function Q11_fun(n)
    Q5_fun(n) && return true # 3
    Q6_fun(n, 3) && return true # 7
    Q6_fun(n, 4) && return true # 11
    hasAncestor(n, isRoot)[2] == 5 && data(parent(n, 2)) isa C && 
        data(parent(n, 4)) isa A && return true # 12
end

Q11 = Query(B, Q11_fun)
A11 = apply(graph, Q11)

4-element Vector{B}:
 B(7)
 B(3)
 B(12)
 B(11)

In [21]:
function Q12_fun(n)
    Q6_fun(n, 3) && return true # 7
    hasAncestor(n, isRoot)[2] == 5 && data(parent(n, 2)) isa C && 
        data(parent(n, 4)) isa A && return true # 12
end

Q12 = Query(B, Q12_fun)
A12 = apply(graph, Q12)

2-element Vector{B}:
 B(7)
 B(12)